In [ ]:
# The Core Problem: LLMs are Isolated Brains.
#  By default, a large language model (LLM) is like a brilliant brain in a jar. It possesses vast amounts of knowledge, but it's
#  static and disconnected from the real world.
#  One major limitation is its knowledge cutoff—its information is frozen at the point when its training data was collected. 
# That means it doesn't know the current date, today's weather, or recent news. 
# Another key limitation is its inability to act. The model cannot perform actions such as sending an email, booking a 
# flight, checking the temperature of your city, or querying a private database. It can only generate text.

# llm is very fluent in text generation and problem solving , resoning and other logical tasks but the majour problem with llm is its cutoff from the real world (knowledge cutoff),
# means llm knows only information till perticular date of the training time .
# llm cant hit the api and give the real time information by its own. 
# ex :- assume i give a email containt and sink email and give the task to llm to send the mail to sink but llm cant send the mail to sink by its own .

# core problems with llm 
# 1. knowledge cutoff 
# 2. not respoding to the current events 
# 3. disconnected from the real world 
# 4. cant perform real world task like doing payments  (inability to act)

In [ ]:
# function calling is the mechanism that gives this brain hands and senses to interact with the outside world
# exernal systems and apis in  sturctured way.
# it is an feature that allows developers to create an function for an specific task, pass it  to the maodel and based 
# on the problem asked use that function and give the output

# define a funtion for get_temprture where input is city and with the help of temprature api we first catch the city temp and 
# then return as an output of city temprature .


# when we call model.GenerateContent() we also pass this tool in this object .

In [ ]:
import os 
os.environ["GOOGLE_API_KEY"]=

In [5]:
from dotenv import load_dotenv
from google import genai
from google.genai import types 

load_dotenv()
api_key=os.getenv("GOOGLE_API_KEY") or os.getenv('GEMINI_API_KEY')

if not api_key :
    raise ValueError("api key is not found")

client = genai.Client(api_key = api_key)


In [8]:
import datetime
import pytz

tz = pytz.timezone("Asia/Kolkata")
print(tz)
now = datetime.datetime.now(tz)
now



Asia/Kolkata


datetime.datetime(2025, 10, 6, 12, 44, 43, 802133, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>)

In [11]:
print(now.strftime("%Y-%m-%d"))
print(now.strftime("%I:%M:%S %p"))

2025-10-06
12:44:43 PM


In [13]:
#whenever we difine the funtion then add (not mandate ) but its helpful to llm to finding right tool to 
# solve the problem .

# type-hinting 
# description 

def get_current_datetime(timezone : str = "UTC") -> dict :
    """fetches the current date and tiime for the given timezone .

        Args:
            timezone(str):it is the timezone name (eg= "Asia/Kolkata","UTC)

        Returns :
            dict: A dictionary containing the current date and time in specified in the specified timezone
            or an error message if the timezone is invalid .

         """
    print(f"--Function get_current_datetime is called for the timezone {timezone} ")

    try :
        tz = pytz.timezone(timezone)
        now = datetime.datetime.now(tz)
        return {
            "date" : now.strftime("%Y-%m-%d"),
            "time" : now.strftime("%I:%M:%S %p")
        }

    except pytz.UnknownTimeZoneError:
        return {"error: invalid timezone specified."}



In [15]:
output = get_current_datetime("Asia/Kolkata")
print(output['date'])
print(output['time'])

--Function get_current_datetime is called for the timezone Asia/Kolkata 
2025-10-06
01:05:23 PM


In [18]:
query ="what is current date and time in chandigarh"

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=query,
    config = types.GenerateContentConfig(
        tools= [get_current_datetime]
    )
    
)

print(response.text)

--Function get_current_datetime is called for the timezone Asia/Kolkata 
The current date and time in Chandigarh is 2025-10-06 01:13:49 PM.


In [17]:
query ="hello"

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=query,
    config = types.GenerateContentConfig(
        tools= [get_current_datetime]
    )
    
)

print(response.text)


Hello! How can I assist you today?



In [ ]:
# here we add thinking_config parameter 

query ="hello"

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=query,
    config = types.GenerateContentConfig(
        tools= [get_current_datetime],
        thinking_config = types.ThinkingConfig(
            include_thoughts=True
        )
    )
    
)
response
# here in the second line - automatic_function_calling_history=[]
# no tool usage when handling the current query "hello"


GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""Hello! How can I assist you today?
"""
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>,
      index=0
    ),
  ],
  model_version='gemini-2.5-flash',
  response_id='xXPjaKHqGNq_1e8Pu_SxyQg',
  sdk_http_response=HttpResponse(
    headers=<dict len=11>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=9,
    prompt_token_count=117,
    prompt_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=117
      ),
    ],
    total_token_count=126
  )
)

In [22]:
query ="what is current date and time in chandigarh"

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=query,
    config = types.GenerateContentConfig(
        tools= [get_current_datetime],
        thinking_config=types.ThinkingConfig(
            include_thoughts=True
        )
    )
    
)

response

--Function get_current_datetime is called for the timezone Asia/Kolkata 


GenerateContentResponse(
  automatic_function_calling_history=[
    UserContent(
      parts=[
        Part(
          text='what is current date and time in chandigarh'
        ),
      ],
      role='user'
    ),
    Content(
      parts=[
        Part(
          text="""**Fetching the Time in Chandigarh**

Okay, so I need to get the current date and time for Chandigarh. I know I have this `get_current_datetime` tool that can do that, but I need to give it a timezone. "Chandigarh" isn't a valid timezone string, so I need to translate that. Let me think... ah, right! "Asia/Kolkata" is the standard timezone for India, and that should be the same for Chandigarh.  Let's call the tool with `timezone='Asia/Kolkata'` and see what we get.
""",
          thought=True
        ),
        Part(
          function_call=FunctionCall(
            args={<... 1 item at Max depth ...>},
            name='get_current_datetime'
          ),
          thought_signature=b'\n\xf2\x04\x01\xd1\xed\x8ao\xc8y\

In [ ]:
 #thinking what llm think when its deal with the query which is releted to time and date or i can say tool
response.automatic_function_calling_history[1].parts[0] 

Part(
  text="""**Fetching the Time in Chandigarh**

Okay, so I need to get the current date and time for Chandigarh. I know I have this `get_current_datetime` tool that can do that, but I need to give it a timezone. "Chandigarh" isn't a valid timezone string, so I need to translate that. Let me think... ah, right! "Asia/Kolkata" is the standard timezone for India, and that should be the same for Chandigarh.  Let's call the tool with `timezone='Asia/Kolkata'` and see what we get.
""",
  thought=True
)

In [26]:
response.automatic_function_calling_history[1].parts[1]  # user . system

# gemini -- function execute(X)
# internal thinking :- function, arguments [value]

# gemini- json schema :- function , arguments , value  ----->system.


# system -- command  -- [gemini : function]  , system

Part(
  function_call=FunctionCall(
    args={
      'timezone': 'Asia/Kolkata'
    },
    name='get_current_datetime'
  ),
  thought_signature=b'\n\xf2\x04\x01\xd1\xed\x8ao\xc8y\xe9\xd7\x82\xf3D\xa70v$6\xa0\xf2\x0c\x18\x0e\xbb\x88\x8f=\xf8\\I\xcd\xacC\x1c8\xae\xa2XV\x1e\xa0>lP\x8b\xbe\x9c\xda\xd2d\xdb\xd5\xe3nj\xea\xb5\x16\x8e=\xc3KE\xf6\xde\x02\xfc<4Z\xe1\xa4\x01y\xf9\xaf\xc9)z\xfeB3\xc3\xff\x93\xe8\x91FS"\x8a\x81\xb8\xf5\xe0...'
)

In [ ]:
response.automatic_function_calling_history[2]

# system -- function ---> o/p ----> i/p -- model
# structured from

Content(
  parts=[
    Part(
      function_response=FunctionResponse(
        name='get_current_datetime',
        response={
          'result': {
            'date': '2025-10-06',
            'time': '01:21:13 PM'
          }
        }
      )
    ),
  ],
  role='user'
)

In [28]:
response.candidates

[Candidate(
   content=Content(
     parts=[
       Part(
         text='The current date and time in Chandigarh is 2025-10-06 01:21:13 PM.'
       ),
     ],
     role='model'
   ),
   finish_reason=<FinishReason.STOP: 'STOP'>,
   index=0
 )]

In [ ]:
# PARALLEL FUNCTION CALLING.
# LLM -- AUTOMATICALLY DECIDE  WHICH FUNCTION TO CALL.
# MULTIPLE : LLM CAN DO.

# USER :- TELL ME CURRENT DATE TIME AND  TEMPERATURE  OF DELHI.

In [30]:
def get_temperature(city: str) -> dict:
    """
    function that returns temperature data for a given city.

    Args:
        city (str): Name of the city

    Returns:
        dict: A dictionary with temperature and condition
    """
    print(f"--- Function get_temperature ({city} Called")
    # Hardcoded fake temperature data
    fake_data = {
        "New York": {"temperature": "30°C", "condition": "Sunny"},
        "Delhi": {"temperature": "36°C", "condition": "Hot and Dry"},
        "London": {"temperature": "22°C", "condition": "Cloudy"},
        "Tokyo": {"temperature": "28°C", "condition": "Humid"},
        "Paris": {"temperature": "25°C", "condition": "Breezy"},
    }

    # Return fake data if city exists, else give default
    return fake_data.get(city, {"temperature": "Unknown", "condition": "Data not available"})

In [31]:
response= client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is the current date and time and temperature for Delhi",
    config= types.GenerateContentConfig(
        tools=[get_current_datetime,get_temperature],
        thinking_config= types.ThinkingConfig(
            include_thoughts=True
        )
    )
)


--Function get_current_datetime is called for the timezone Asia/Kolkata 
--- Function get_temperature (Delhi Called


In [32]:
response

GenerateContentResponse(
  automatic_function_calling_history=[
    UserContent(
      parts=[
        Part(
          text='What is the current date and time and temperature for Delhi'
        ),
      ],
      role='user'
    ),
    Content(
      parts=[
        Part(
          text="""**Processing the User's Request**

Okay, the user needs two things: the date and time in Delhi, and the current temperature there.  No problem.  I'll start by grabbing the current date and time using `get_current_datetime`. I need to remember to specify the "Asia/Kolkata" timezone since that's where Delhi is located. After that, I can use `get_temperature` to fetch the current temperature for "Delhi".  Once I have both pieces of information, I can combine them into a coherent response for the user. Easy enough!
""",
          thought=True
        ),
        Part(
          function_call=FunctionCall(
            args={<... 1 item at Max depth ...>},
            name='get_current_datetime'
          ),